In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [17]:
df = pd.read_csv("data/spotify_millsongdata_w_genres.csv")

In [18]:
song_embeddings = model.encode(df['song'].values, show_progress_bar=True)
lyrics_embeddings = model.encode(df['cleaned_text'].values, show_progress_bar=True)
genres_embeddings = model.encode(df['genres'].values, show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [19]:
embeddings = []
for i in range(len(song_embeddings)):
    embeddings.append((song_embeddings[i] + lyrics_embeddings[i] + genres_embeddings[i]) / 3)

In [20]:
cos_sim = util.cos_sim(embeddings, embeddings)

In [21]:
cos_sim.shape

torch.Size([100, 100])

In [22]:
similar_songs = np.flip(np.argsort(cos_sim[10].numpy()))[1:10]

In [23]:
similar_songs

array([11, 74, 14, 91, 95, 22, 96, 84,  3], dtype=int64)

In [26]:
df.loc[10]

artist                                                       ABBA
song                                                        Dance
cleaned_text    oh , my love it makes me sad . why did things ...
genres                                                        Pop
Name: 10, dtype: object

In [25]:
df.loc[similar_songs]

,artist,song,cleaned_text,genres
11,ABBA,Dancing Queen,"you can dance , you can jive , having the time...","Pop,Dance"
74,ABBA,Reina Danzante,you can dance you can jive having the time of ...,"Pop,Dance"
14,ABBA,Dream World,agnetha we 're not the stars of a hollywood mo...,"Dance,Pop"
91,ABBA,Thank You For The Music,"i 'm nothing special , in fact i 'm a bit of a...",Pop
95,ABBA,The Name Of The Game,"i 've seen you twice , in a short time only a ...","Pop,Dance"
22,ABBA,Gimme Gimme Gimme,half past twelve and i 'm watching the late sh...,"Pop,Rock"
96,ABBA,The Piper,they came from the hills and they came from th...,"Rock,Pop"
84,ABBA,So Long,"you think you 're gon na make me sulk , you an...","Rock,Pop"
3,ABBA,Bang,making somebody happy is a question of give an...,Pop
